In [5]:
import numpy as np
import pandas as pd
import os

import pathlib
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

In [8]:
root = './data/cleaning_data_95.csv'
data = pd.read_csv(root)

In [9]:
data.head()

,item,labels,subject,topic
0,加 试题 阅读 材料 完成 下列 问题 材料 一辽 中南 工业区 左图 北部湾 经济区 右图...,高中 地理 宇宙中的地球 工业区位因素 地球运动的地理意义,地理,宇宙中的地球
1,精子 变形 叙述 错误 线粒体 退化 消失 高尔基体 发育 头部 顶体 中心 体 演变 精子...,高中 生物 现代生物技术专题 生物技术在其他方面的应用,生物,现代生物技术专题
2,哈尔滨 北京 上海 广州 四地 自转 速度 关系 自转 线速度 相同 自转角 速度 相同 线...,高中 地理 宇宙中的地球 地球运动的基本形式,地理,宇宙中的地球
3,太阳活动 高峰期 时 色球层 上 太阳黑子 增多 光球 层上 耀斑 大 爆发 黑子 活动 耀...,高中 地理 宇宙中的地球 太阳对地球的影响,地理,宇宙中的地球
4,年月日 国家 主席 习近平 伦敦 唐宁街 首相府 英国首相 卡梅伦 举行会谈 双方 决定 共...,高中 历史 现代史 海峡两岸关系的发展,历史,现代史


In [10]:
corpus=data['item']

from sklearn.feature_extraction.text import TfidfVectorizer
#使用tfidf统计词频特征 来作为文本特征
tfidf_vect = TfidfVectorizer(max_features=20000,min_df=3,ngram_range=(2,4))
vec = tfidf_vect.fit_transform(corpus)

print('词汇表为：\n', len(tfidf_vect.vocabulary_))

词汇表为：
 20000


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
mlb = MultiLabelBinarizer()
lb = LabelEncoder()

### 4分类

In [14]:
y_4 = lb.fit_transform(data['subject'])

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(vec,y_4,test_size = 0.2,random_state = 2020)

In [17]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB,BernoulliNB

In [18]:
gnb = GaussianNB()        #高斯分布贝叶斯    假设x的先验概率服从高斯分布
clf = MultinomialNB()     #多项式分布贝叶斯  假设x的先验概率服从多项式分布
bnb = BernoulliNB()       #伯努利分布贝叶斯  假设x的先验概率服从二元伯努利分布
cnb = ComplementNB()      #补充朴素贝叶斯    Mnb的改编，适用于不平衡数据集

In [22]:
%%time
gnb.fit(x_train.toarray(),y_train)  #高斯的输入需转换为np数组

clf.fit(x_train,y_train)

bnb.fit(x_train,y_train)

cnb.fit(x_train,y_train)

Wall time: 12.7 s


In [23]:
##预测
pre1 = gnb.predict(x_test.toarray())
pre2 = clf.predict(x_test)
pre3 = bnb.predict(x_test)
pre4 = cnb.predict(x_test)

In [26]:
from sklearn.metrics import classification_report
print('gnb Result：',classification_report(y_test,pre1))
print('clf Result：',classification_report(y_test,pre2))
print('bnb Result：',classification_report(y_test,pre3))
print('cnb Result：',classification_report(y_test,pre4))

gnb Result：               precision    recall  f1-score   support

           0       0.91      0.94      0.92       532
           1       0.98      0.94      0.96       978
           2       0.93      0.98      0.95       371
           3       0.99      0.99      0.99      2635

    accuracy                           0.97      4516
   macro avg       0.95      0.96      0.96      4516
weighted avg       0.97      0.97      0.97      4516

clf Result：               precision    recall  f1-score   support

           0       0.00      0.00      0.00       532
           1       0.94      0.05      0.09       978
           2       0.97      0.18      0.30       371
           3       0.60      1.00      0.75      2635

    accuracy                           0.61      4516
   macro avg       0.63      0.31      0.28      4516
weighted avg       0.63      0.61      0.48      4516

bnb Result：               precision    recall  f1-score   support

           0       0.55      0.99      

### 对比一下其他分类器

**LR**

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=2)
lr.fit(x_train,y_train)
pre = lr.predict(x_test)
print('Result：',classification_report(y_test,pre))

Result：               precision    recall  f1-score   support

           0       0.79      0.18      0.29       532
           1       0.99      0.38      0.55       978
           2       0.97      0.33      0.49       371
           3       0.68      1.00      0.81      2635

    accuracy                           0.71      4516
   macro avg       0.86      0.47      0.53      4516
weighted avg       0.78      0.71      0.66      4516



**SVM**

In [30]:
%%time
from sklearn.svm import SVC
svc = SVC(kernel='rbf',C=1)
svc.fit(x_train,y_train)
pre = svc.predict(x_test)
print('Result：',classification_report(y_test,pre))

Result：               precision    recall  f1-score   support

           0       0.82      0.18      0.29       532
           1       0.99      0.34      0.50       978
           2       0.97      0.43      0.60       371
           3       0.67      1.00      0.80      2635

    accuracy                           0.71      4516
   macro avg       0.86      0.49      0.55      4516
weighted avg       0.78      0.71      0.66      4516

Wall time: 34.3 s


**RandomForest**

In [31]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
pre = rf.predict(x_test)
print('Result：',classification_report(y_test,pre))

Result：               precision    recall  f1-score   support

           0       0.82      0.88      0.85       532
           1       0.94      0.86      0.90       978
           2       0.97      0.81      0.88       371
           3       0.94      0.98      0.96      2635

    accuracy                           0.93      4516
   macro avg       0.92      0.88      0.90      4516
weighted avg       0.93      0.93      0.93      4516

Wall time: 10.7 s


**XGboost**

In [32]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(x_train,y_train)
pre = model.predict(x_test)
print('Result：',classification_report(y_test,pre))

Result：               precision    recall  f1-score   support

           0       0.77      0.67      0.72       532
           1       0.91      0.52      0.66       978
           2       0.97      0.50      0.66       371
           3       0.78      0.98      0.87      2635

    accuracy                           0.80      4516
   macro avg       0.86      0.67      0.73      4516
weighted avg       0.82      0.80      0.79      4516



#### 对比几种分类器，在4分类的情况下，GaussianNB的分类效果最好，其次是RandomForest